In [1]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

In [2]:
from funcat import *

In [3]:
import time
import datetime
n1 = (time.strftime("20%y-%m-%d"))
n2 = (time.strftime("20%y%m%d"))
#print (time.strftime("20%y-%m-%d")) ## dd/mm/yyyy格式，输出当日日期
#print (time.strftime("20%y%m%d"))
def getYesterday(): 
    today=datetime.date.today() 
    oneday=datetime.timedelta(days=1) 
    yesterday=today-oneday  
    return yesterday
n3 = (getYesterday())
#print(getYesterday())# 输出前一天日期
i = datetime.datetime.now()
#print ("%s" % i)
#print ("%s" % i.isoformat() )
#print ("%s%s%s" % (i.year, i.month, i.day) )
n4 = ("%s%s%s" % (i.year, i.month, i.day) )
n5 = ("%s" % i.isoformat() )
print ("读取当日时间格式n1",n1) #当日时间格式：yyyy-mm-dd
print ("读取当日时间格式n2",n2) #当日时间格式：yyyymmdd
print ("读取前一日时间格式n3",n3) #前日时间格式：yyyy-mm-dd
#print ("读取当日时间格式n4",n4) #当日时间格式：yyyymmdd
#print ("读取当日时间格式n5",n5) #当日时间格式：yyyy-mm-dd:...

n8="20180130"#开始时间

print ("手动设定选股开始时间格式n8",n8)

n9="20180130"#结束时间

print ("手动设定选股结束时间格式n9",n9)

n10="20170101"#设定起始时间

print ("手动设定数据更新设定起始时间格式n10",n10)

读取当日时间格式n1 2018-01-31
读取当日时间格式n2 20180131
读取前一日时间格式n3 2018-01-30
手动设定选股开始时间格式n8 20180130
手动设定选股结束时间格式n9 20180130
手动设定数据更新设定起始时间格式n10 20170101


In [4]:
from funcat.data.tushare_backend import TushareDataBackend
from funcat.data.rqalpha_data_backend import RQAlphaDataBackend

backend = "tushare"

if backend == "rqalpha":
    set_data_backend(RQAlphaDataBackend("~/.rqalpha/bundle"))
elif backend == "tushare":
    set_data_backend(TushareDataBackend())

set_start_date(n10)#数据同步起始日
S("000001.XSHG")  # 设置当前关注股票
T(n9) # 设置当前观察日期


## 基本API

In [5]:
print(O, H, L, C, V)  # 打印 Open High Low Close

3511.5 3523.05 3484.66 3488.01 186369975.0


In [6]:
C / C[1] - 1  # 当天涨幅

-0.009931876241839332

In [7]:
MA(C, 60)  # 打印60日均线

3379.029666666665

In [8]:
C > MA(C, 60)  # 判断收盘价是否大于60日均线

True

In [9]:
HHV(H, 30)  # 30日最高价

3587.03

In [10]:
COUNT(C > MA(C, 60), 30)  # 最近30日，收盘价 Close 大于60日均线的天数

20

In [11]:
CROSS(MA(C, 10), MA(C, 20))  # 10日均线上穿

False

In [12]:
COUNT(C > O, 10)  # 最近10天收阳线的天数

7

In [13]:
HHV(MAX(O, C), 60)  # 最近60天K线实体的最高价

3563.64

In [14]:
CROSS(MA(C, 5), MA(C, 10))  # 5日均线上穿10日均线

False

## 自定义公式

### 0x01 KDJ指标

In [15]:
N, M1, M2 = 27, 9, 3

RSV = (CLOSE - LLV(LOW, N)) / (HHV(HIGH, N) - LLV(LOW, N)) * 100
K = EMA(RSV, (M1 * 2 - 1))
D = EMA(K, (M2 * 2 - 1))
J = K * 3 - D * 2

ax = plt.subplot()

plt.plot(K.series, label="K", linewidth=2)
plt.plot(D.series, label="D", alpha=0.7)
plt.plot(J.series, label="J", alpha=0.7)

print(K, D, J)
plt.legend(loc="best")

84.70517755686883 85.17444544261426 83.766641785378


### 0x02 均线金叉死叉

In [16]:
ax = plt.subplot()

ma10 = MA(C, 10)
ma30 = MA(C, 30)
buy_signal = CROSS(ma10, ma30)
sell_signal = CROSS(ma30, ma10)

plt.plot(C.series, label="close", linewidth=2)
plt.plot(ma10.series, label="ma10", alpha=0.7)
plt.plot(ma30.series, label="ma30", alpha=0.7)

plt.plot(np.where(buy_signal.series)[0], C.series[np.where(buy_signal.series)[0]], "^", label="buy", markersize=12, color="red")
plt.plot(np.where(sell_signal.series)[0], C.series[np.where(sell_signal.series)[0]], "v", label="sell", markersize=12, color="green")
plt.legend(loc="best")

/Users/zy/anaconda3/envs/funcat/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


### 0x03 DMA指标（Different of Moving Average）
[DMA指标](http://wiki.mbalib.com/wiki/DMA)。DMA指标（Different of Moving Average）又叫平行线差指标，是目前股市分析技术指标中的一种中短期指标，它常用于大盘指数和个股的研判。

In [ ]:
M1 = 10
M2 = 50
M3 = 10

DDD = MA(CLOSE, M1) - MA(CLOSE, M2)
AMA = MA(DDD, M3)

print(DDD, AMA)

f, (ax1, ax2) = plt.subplots(2, 1)

ax1.plot(C.series, label="close")
ax1.plot(MA(C, 30).series, label="ma60")
ax1.set_xlim(50)

ax2.plot(DDD.series, label="DDD")
ax2.plot(AMA.series, label="AMA")
ax2.set_xlim(50)

plt.legend(loc="best")

141.2496000000001 104.2402000000001


## 选股

In [ ]:
# 选出涨停股
select(
    lambda : C / C[1] - 1 >= 0.0995,
    start_date=(n8),
	end_date=(n9),
)

lambda : C / C[1] - 1 >= 0.0995,
[20180130]
20180130 000505.XSHE 000505.XSHE[珠江控股]
20180130 000836.XSHE 000836.XSHE[鑫茂科技]
20180130 002016.XSHE 002016.XSHE[世荣兆业]
20180130 002192.XSHE 002192.XSHE[融捷股份]
20180130 002229.XSHE 002229.XSHE[鸿博股份]
20180130 002329.XSHE 002329.XSHE[皇氏集团]
20180130 002544.XSHE 002544.XSHE[杰赛科技]
20180130 002696.XSHE 002696.XSHE[百洋股份]
20180130 002805.XSHE 002805.XSHE[丰元股份]
20180130 002925.XSHE 002925.XSHE[盈趣科技]
20180130 600158.XSHG 600158.XSHG[中体产业]
20180130 600238.XSHG 600238.XSHG[海南椰岛]
20180130 600506.XSHG 600506.XSHG[香梨股份]
20180130 600821.XSHG 600821.XSHG[津劝业]
20180130 600838.XSHG 600838.XSHG[上海九百]


In [ ]:
# 选出最近30天K线实体最高价最低价差7%以内，最近100天K线实体最高价最低价差25%以内，
# 最近10天，收盘价大于60日均线的天数大于3天
select(
    lambda : ((HHV(MAX(C, O), 30) / LLV(MIN(C, O), 30) - 1 < 0.07)
              & (HHV(MAX(C, O), 100) / LLV(MIN(C, O), 100) - 1 > 0.25)
              & (COUNT(C > MA(C, 60), 10) > 3)
             ),
    start_date=(n8),
	end_date=(n9),
)

In [ ]:
# 选出最近3天每天的成交量小于20日成交量均线，最近3天最低价低于20日均线，最高价高于20日均线
# 自定义选股回调函数
def callback(date, order_book_id, symbol):
    print("Cool, 在", date, "选出", order_book_id, symbol)

    
select(
    lambda : (EVERY(V < MA(V, 20) / 2, 3) & EVERY(L < MA(C, 20), 3) & EVERY(H > MA(C, 20), 3)),
    start_date=(n8),
	end_date=(n9),
    callback=callback,
)